In [35]:
import pandas as pd
import numpy as np
from geopy.geocoders import Nominatim
import requests

In [2]:
def get_latitude_longitude(city):
    geolocator = Nominatim(user_agent="my-application")
    location = geolocator.geocode(city)
    return location.latitude, location.longitude

In [3]:
df = pd.DataFrame({"city":["New York City, NY",
                           "Los Angeles, CA",
                           "Chicago, IL",
                           "Houston, TX",
                           "Philadelphia, PA",
                           "Phoenix, AZ",
                           "San Antonio, TX",
                           "San Diego, CA",
                           "Dallas, TX",
                           "San Jose, CA"]})
df['latitude'],df['longitude'] = zip(*df['city'].apply(get_latitude_longitude))
df

,city,latitude,longitude
0,"New York City, NY",40.712728,-74.006015
1,"Los Angeles, CA",34.053691,-118.242767
2,"Chicago, IL",41.875562,-87.624421
3,"Houston, TX",29.758938,-95.367697
4,"Philadelphia, PA",39.952724,-75.163526
5,"Phoenix, AZ",33.448587,-112.077346
6,"San Antonio, TX",29.424600,-98.495141
7,"San Diego, CA",32.717421,-117.162771
8,"Dallas, TX",32.776272,-96.796856
9,"San Jose, CA",37.336191,-121.890583


In [4]:
# @hidden_cell
CLIENT_ID = 'CRDS4XWDD4KPADKBVJP1JE5NW3AYATHVMUCCUC4ZFNIWNJZ5' # your Foursquare ID
CLIENT_SECRET = 'ZBKR4ETVK14UV2SMZRJ1OCPSLAZFF2K2RPBEWFEKWHFVFHTE' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

In [24]:
def getNearbyVenues(names, latitudes, longitudes, radius=10000,LIMIT=100):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius,
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['City', 
                  'City Latitude', 
                  'City Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [25]:
city_venues = getNearbyVenues(df['city'],df['latitude'],df['longitude'])


In [26]:
city_venues['City'].value_counts()

Chicago, IL          100
Philadelphia, PA     100
San Diego, CA        100
San Jose, CA         100
New York City, NY    100
San Antonio, TX      100
Los Angeles, CA      100
Dallas, TX           100
Houston, TX          100
Phoenix, AZ          100
Name: City, dtype: int64

In [30]:
# one hot encoding
city_onehot = pd.get_dummies(city_venues[['Venue Category']], prefix="", prefix_sep="")
city_onehot['City'] = city_venues['City']
fixed_columns = [city_onehot.columns[-1]] + list(city_onehot.columns[:-1])
city_onehot = city_onehot[fixed_columns]

city_onehot.head()

,City,American Restaurant,Amphitheater,Antique Shop,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,...,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Waterfront,Whisky Bar,Wine Bar,Wine Shop,Winery,Yoga Studio,Zoo,Zoo Exhibit
0,"New York City, NY",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"New York City, NY",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"New York City, NY",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"New York City, NY",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"New York City, NY",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [31]:
city_onehot_mean = city_onehot.groupby('City').mean()
city_onehot_mean

,American Restaurant,Amphitheater,Antique Shop,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Automotive Shop,...,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Waterfront,Whisky Bar,Wine Bar,Wine Shop,Winery,Yoga Studio,Zoo,Zoo Exhibit
City,,,,,,,,,,,,,,,,,,,,,
"Chicago, IL",0.02,0.01,0.00,0.00,0.00,0.01,0.00,0.01,0.00,0.00,...,0.00,0.00,0.02,0.00,0.00,0.00,0.00,0.02,0.00,0.0
"Dallas, TX",0.02,0.00,0.00,0.00,0.00,0.03,0.00,0.01,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.01,0.00,0.0
"Houston, TX",0.02,0.00,0.00,0.00,0.01,0.02,0.01,0.00,0.00,0.00,...,0.00,0.03,0.00,0.00,0.02,0.00,0.00,0.01,0.00,0.0
"Los Angeles, CA",0.02,0.00,0.00,0.00,0.03,0.01,0.00,0.01,0.00,0.00,...,0.00,0.01,0.00,0.00,0.00,0.00,0.01,0.00,0.00,0.0
"New York City, NY",0.01,0.00,0.00,0.00,0.01,0.00,0.00,0.00,0.01,0.00,...,0.00,0.00,0.00,0.00,0.01,0.02,0.00,0.03,0.00,0.0
"Philadelphia, PA",0.02,0.00,0.00,0.00,0.00,0.02,0.01,0.01,0.00,0.00,...,0.02,0.00,0.00,0.00,0.03,0.00,0.00,0.00,0.00,0.0
"Phoenix, AZ",0.02,0.00,0.01,0.01,0.01,0.01,0.00,0.00,0.00,0.00,...,0.02,0.00,0.00,0.00,0.00,0.00,0.00,0.01,0.00,0.0
"San Antonio, TX",0.05,0.00,0.00,0.00,0.00,0.01,0.01,0.00,0.00,0.00,...,0.02,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0
"San Diego, CA",0.03,0.01,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.02,0.00,0.00,0.01,0.01,0.00,0.00,0.00,0.02,0.1


In [36]:
x=pd.DataFrame(city_onehot_mean).T
rslt = pd.DataFrame(np.zeros((0,5)), columns=['top1','top2','top3','top4','top5'])
for i in x.columns:
    df1row = pd.DataFrame(x.nlargest(5, i).index.tolist(), index=['top1','top2','top3','top4','top5']).T
    rslt = pd.concat([rslt, df1row], axis=0)

rslt.index=city_onehot_mean.index
rslt

,top1,top2,top3,top4,top5
City,,,,,
"Chicago, IL",Hotel,Park,New American Restaurant,Theater,Boat or Ferry
"Dallas, TX",Coffee Shop,Hotel,Bar,Park,Art Museum
"Houston, TX",Coffee Shop,Beer Garden,Café,Mexican Restaurant,Park
"Los Angeles, CA",Coffee Shop,Brewery,Ice Cream Shop,Theater,Taco Place
"New York City, NY",Park,Bakery,Bookstore,Cycle Studio,Gym
"Philadelphia, PA",Coffee Shop,Pizza Place,Hotel,Indian Restaurant,New American Restaurant
"Phoenix, AZ",Pizza Place,Mexican Restaurant,Coffee Shop,Bar,Breakfast Spot
"San Antonio, TX",Hotel,American Restaurant,Cocktail Bar,Theater,Beer Garden
"San Diego, CA",Zoo Exhibit,Park,Hotel,Mexican Restaurant,Seafood Restaurant
